In [1]:
import pandas as pd
import sqlalchemy
from time import time
from pathlib import Path
import os

In [2]:
user = 'root'
password = 'root'
host = 'localhost'
port = '5432'
database = 'ny_taxi'
table = 'green_taxi_rides'
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}'
                        f'@{host}:{port}/{database}')

In [3]:
two_levels_up = Path(os.getcwd()).parents[1].as_posix()
file_path = two_levels_up + '/nyc_cab_csv/green_tripdata_2019-01.csv.gz'

In [4]:
# Create an iterator so we can read by chunks
df_iter = pd.read_csv(file_path, iterator=True, chunksize = 100_000,
compression='gzip',)
i = 0
for df in df_iter:
    i += 1
    t_start = time()
    date_time_fields = ['lpep_pickup_datetime','lpep_dropoff_datetime', #for green taxi
                        'tpep_pickup_datetime', 'tpep_dropoff_datetime' ] # for yellow taxi
    for dt_field in date_time_fields:
        if dt_field in df.columns:
            df[dt_field] = pd.to_datetime(df[dt_field])
    df.to_sql(name = table, con = engine, if_exists = 'append')
    t_end = time()
    print(f'inserted chunK nro {i} with {df.shape[0]} rows. Took {t_end - t_start} seconds.')

inserted chunK nro 1 with 100000 rows. Took 8.045594453811646 seconds.
inserted chunK nro 2 with 100000 rows. Took 7.808966636657715 seconds.
inserted chunK nro 3 with 100000 rows. Took 7.759404182434082 seconds.
inserted chunK nro 4 with 100000 rows. Took 7.796630382537842 seconds.
inserted chunK nro 5 with 100000 rows. Took 8.073123216629028 seconds.
inserted chunK nro 6 with 100000 rows. Took 8.066332578659058 seconds.
inserted chunK nro 7 with 30918 rows. Took 2.48249888420105 seconds.


In [5]:
zones_file_path = file_path = two_levels_up + '/nyc_cab_csv/taxi+_zone_lookup.csv'
df_zones =  pd.read_csv(zones_file_path)
df_zones.to_sql(name = 'taxi_zones_lookup', con = engine, if_exists = 'replace')

265

### Question 3. Count records

How many taxi trips were totally made on January 15?

In [6]:

query = '''
SELECT COUNT(*)
FROM green_taxi_rides
WHERE DATE(lpep_pickup_datetime) = '2019-01-15'
AND DATE(lpep_dropoff_datetime) = '2019-01-15'
'''
pd.read_sql(query, con = engine)

,count
0,20530


### Question 4. Largest trip for each day

Which was the day with the largest trip distance Use the pick up time for your calculations.

In [7]:
query = '''
SELECT DATE(lpep_pickup_datetime) AS date, trip_distance
FROM green_taxi_rides
ORDER BY trip_distance DESC
LIMIT 3
'''
pd.read_sql(query, con = engine)

,date,trip_distance
0,2019-01-15,117.99
1,2019-01-18,80.96
2,2019-01-28,64.27


### Question 5. The number of passengers

In 2019-01-01 how many trips had 2 and 3 passengers?

In [8]:
query = '''
SELECT passenger_count, COUNT(*)
FROM green_taxi_rides
WHERE passenger_count BETWEEN 2 AND 3
AND DATE(lpep_pickup_datetime) = '2019-01-01'
GROUP BY passenger_count
'''
pd.read_sql(query, con = engine)

,passenger_count,count
0,2,1282
1,3,254


### Question 6. Largest tip

For the passengers picked up in the Astoria Zone which was the drop off zone that had the largest tip? We want the name of the zone, not the id.

In [9]:
query = '''
SELECT do_zone."Zone" as dropoff_zone,
COUNT(*) AS q_trips, MAX(tip_amount) as max_tip
FROM green_taxi_rides gtr
LEFT JOIN taxi_zones_lookup pu_zone
    ON gtr."PULocationID" = pu_zone."LocationID"
LEFT JOIN taxi_zones_lookup do_zone
    ON gtr."DOLocationID" = do_zone."LocationID"
WHERE pu_zone."Zone" = 'Astoria'
GROUP BY do_zone."Zone"
ORDER BY max_tip DESC
LIMIT 5
'''
pd.read_sql(query, con = engine)

,dropoff_zone,q_trips,max_tip
0,Long Island City/Queens Plaza,1105,88.00
1,Central Park,36,30.00
2,Jamaica,45,25.00
3,None,14,25.00
4,Astoria,7829,18.16
